# Manejo de missing values

Source: https://machinelearningmastery.com/handle-missing-data-python/

## Diabetes Dataset

Estructura del dataset:

0. Number of times pregnant.
1. Plasma glucose concentration a 2 hours in an oral glucose tolerance test.
2. Diastolic blood pressure (mm Hg).
3. Triceps skinfold thickness (mm).
4. 2-Hour serum insulin (mu U/ml).
5. Body mass index (weight in kg/(height in m)^2).
6. Diabetes pedigree function.
7. Age (years).
8. Class variable (0 or 1).

In [1]:
from sklearn import datasets
import pandas as pd
from numpy import nan
from numpy import isnan
from sklearn.impute import SimpleImputer

In [2]:
columns = ['timesPregnant', 'plasmaGlucose', 'bloodPressure ', 'skinfoldThickness', 'insulin', 'bmi', 'diabetesPedigree', 'age', 'class']
xDataset = pd.read_csv("data/pima-indians-diabetes.csv", header = None, names = columns)
xDataset

,timesPregnant,plasmaGlucose,bloodPressure,skinfoldThickness,insulin,bmi,diabetesPedigree,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


Por el conocimiento del dominio se sabe que este conjunto de datos tiene valores perdidos. Específicamente, faltan observaciones para algunas columnas que están marcadas como un valor cero. Por ej. un cero para el índice de masa corporal (bmi) o la presión arterial (bloodPressure) no es válido.

En este caso, los valores de 0 en esas columnas pueden indicar missing values.

In [3]:
xDataset.describe()

,timesPregnant,plasmaGlucose,bloodPressure,skinfoldThickness,insulin,bmi,diabetesPedigree,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


La mayoría de los datos tienen valores perdidos y la probabilidad de tener valores perdidos aumenta con el tamaño del conjunto de datos. Los datos faltantes no son raros en conjuntos de datos reales. De hecho, la probabilidad de que falte al menos un punto de datos aumenta a medida que aumenta el tamaño del conjunto de datos.


## Mark Missing Values 

Los missing values se indican mediante:

- Entradas fuera de rango: quizás un número negativo (por ejemplo, -1) en un campo numérico que normalmente es solo positivo, o un 0 en un campo numérico que normalmente nunca puede ser 0. Cuando tenemos este caso, es necesario marcar los missing values.

- En Python, específicamente Pandas, NumPy y Scikit-Learn, marcamos los valores faltantes como NaN. En este caso, no es necesario aplicar este paso de marcar los missing values. 

Una vez que sabemos cuáles son los features con nan, podemos tomar la decisión de eliminar las observaciones/columnas o aplicar métodos de imputación para reemplazar los valores perdidos.


En el dataset analizado, las siguientes columnas tienen un valor mínimo cero no válido, por tanto, procedemos a marcarlos:

1. Plasma glucose concentration a 2 hours in an oral glucose tolerance test.
2. Diastolic blood pressure (mm Hg).
3. Triceps skinfold thickness (mm).
4. 2-Hour serum insulin (mu U/ml).
5. Body mass index (weight in kg/(height in m)^2).


Para las 5 features no hay valores nan pero hay 0s lo cual no es un valor válido, por tanto, a esos valores los debemos marcar de forma explícita mediante nan.


In [4]:
xDataset.head(20) # exploramos los datos antes de aplicar alguna decisión.

,timesPregnant,plasmaGlucose,bloodPressure,skinfoldThickness,insulin,bmi,diabetesPedigree,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


Ejecutando el ejemplo, podemos ver claramente 0s en las columnas: 'plasmaGlucose', 'bloodPressure ', 'skinfoldThickness', 'insulin', 'bmi'


Podemos marcar valores como NaN fácilmente con Pandas DataFrame usando la función replace() en un subconjunto de las columnas que nos interesan. 

A continuación se muestra el mismo ejemplo, excepto que imprimimos las primeras 20 filas de datos.

In [5]:
from numpy import nan

# variables con missing values:
columnsMV = ['plasmaGlucose', 'bloodPressure ', 'skinfoldThickness', 'insulin', 'bmi']


# replace '0' values with 'nan'
xMV = xDataset
xMV[columnsMV] = xMV[columnsMV].replace(0, nan)

# visualizar valores transformados:
print(xMV.head(20))




    timesPregnant  plasmaGlucose  bloodPressure   skinfoldThickness  insulin  \
0               6          148.0            72.0               35.0      NaN   
1               1           85.0            66.0               29.0      NaN   
2               8          183.0            64.0                NaN      NaN   
3               1           89.0            66.0               23.0     94.0   
4               0          137.0            40.0               35.0    168.0   
5               5          116.0            74.0                NaN      NaN   
6               3           78.0            50.0               32.0     88.0   
7              10          115.0             NaN                NaN      NaN   
8               2          197.0            70.0               45.0    543.0   
9               8          125.0            96.0                NaN      NaN   
10              4          110.0            92.0                NaN      NaN   
11             10          168.0        

Una vez que hayamos marcado los valores faltantes, podemos usar la función isnull() para contar.

In [6]:
# contar el número de valores nan en cada columna:
print(40*'-')
print(xMV.isnull().sum()) # resumen de valores nulos

# contar el número de valores nan en cada columna ordenando desde la feature con mayor cantidad de nan:
xMV.isnull().sum().sort_values(ascending=False)/len(xMV)*100

----------------------------------------
timesPregnant          0
plasmaGlucose          5
bloodPressure         35
skinfoldThickness    227
insulin              374
bmi                   11
diabetesPedigree       0
age                    0
class                  0
dtype: int64


insulin              48.697917
skinfoldThickness    29.557292
bloodPressure         4.557292
bmi                   1.432292
plasmaGlucose         0.651042
class                 0.000000
age                   0.000000
diabetesPedigree      0.000000
timesPregnant         0.000000
dtype: float64

Podemos ver que las columnas plasmaGlucose, bmi y bloodPressure tienen solo unos pocos valores missing, mientras que las columnas insulin y skinfoldThickness tienen muchos valores nan.

Esto destaca que pueden ser necesarias diferentes estrategias de "missing values" para diferentes features, el objetivo es asegurarnos de que nos quedan suficientes observaciones para crear un modelo analítico.

Los valores con un valor NaN se ignoran de operaciones como suma, recuento, etc. pero en otras operaciones, nos puede generar un error.

Tener valores faltantes en un conjunto de datos puede causar errores con algunos algoritmos de aprendizaje automático. Varios modelos predictivos populares, como SVM,  y las redes neuronales, no pueden tolerar ninguna cantidad de valores perdidos.


## Eliminar filas con valores perdidos

La estrategia más sencilla para manejar los datos faltantes es eliminar los registros que contienen un valor perdido.

El enfoque más simple para tratar con valores perdidos es eliminar predictores completos y / o muestras que contienen valores perdidos.


Podemos hacer esto creando un nuevo Pandas DataFrame con las filas que contienen los valores faltantes eliminados.

Pandas proporciona la función dropna () que se puede utilizar para eliminar columnas o filas con datos faltantes. Podemos usar dropna () para eliminar todas las filas con datos faltantes, de la siguiente manera:

In [7]:
# tamaño del dataset original:
print(xMV.shape)

# borrar filas
xMV.dropna(inplace=True)
# tamaño del conjunto reducido
print(xMV.shape)

(768, 9)
(392, 9)


Al ejecutar este ejemplo, podemos ver que el número de filas se ha reducido fuertemente de 768 en el conjunto de datos original a 392 con todas las filas que contienen un NaN eliminado.

Cuando la cantidad de observaciones que se eliminarían es considerable, podríamos optar por imputar los valores perdidos. 

## Imputar valores perdidos

La imputación se refiere al uso de un modelo para reemplazar los valores perdidos. Los datos faltantes pueden imputarse. En este caso, podemos usar información en las otras variables.

Para reemplazar podemos utilizar:


- Un valor medio, mediano o de moda para la columna.
- Un valor estimado por otro modelo predictivo.

Cualquier método utilizado deberá aplicarse a nuevos datos. Por ejemplo, si elegimos imputar con valores medios de columna, estos valores medios deberán almacenarse en el archivo para su uso posterior en nuevos datos que tengan valores perdidos.

Pandas proporciona la función fillna () para reemplazar los valores faltantes con un valor específico. Por ejemplo, podemos usar fillna () para reemplazar los valores faltantes con el valor medio de cada columna, de la siguiente manera:

In [8]:
# completar los missing values con el valor medio
xMV.fillna(xMV.mean(), inplace=True)
# contar el número de NaN values en cada feature
print(xMV.isnull().sum()) # recuento del número de valores perdidos en cada columna, mostrando cero valores perdidos.


timesPregnant        0
plasmaGlucose        0
bloodPressure        0
skinfoldThickness    0
insulin              0
bmi                  0
diabetesPedigree     0
age                  0
class                0
dtype: int64


In [9]:
xMV.head() # ya no están los valores nan

,timesPregnant,plasmaGlucose,bloodPressure,skinfoldThickness,insulin,bmi,diabetesPedigree,age,class
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
6,3,78.0,50.0,32.0,88.0,31.0,0.248,26,1
8,2,197.0,70.0,45.0,543.0,30.5,0.158,53,1
13,1,189.0,60.0,23.0,846.0,30.1,0.398,59,1



- Imputación mediante SimpleImputer:

La biblioteca scikit-learn proporciona la clase de preprocesamiento SimpleImputer que se puede usar para reemplazar los valores perdidos (con el valor medio, mediana o moda). Esta opción se puede utilizar en lugar de fillna().

Es una clase flexible que nos permite especificar el valor a reemplazar y la técnica utilizada para reemplazarlo (como media, mediana o moda). La clase SimpleImputer opera directamente en la matriz NumPy en lugar del DataFrame.

El siguiente ejemplo usa la clase SimpleImputer para reemplazar los valores faltantes con la media de cada columna y luego imprime el número de valores de NaN en la matriz transformada.

In [10]:
# almacenar valores en un numpy array
values = xDataset.values  # dataframe original

# definir el imputer
imputer = SimpleImputer(strategy='mean')

# transformar el dataset
transformed_values = imputer.fit_transform(values)

# contar el número of NaN en cada columna
print('Missing: %d' % isnan(transformed_values).sum())
transformed_values

Missing: 0


array([[  1.   ,  89.   ,  66.   , ...,   0.167,  21.   ,   0.   ],
       [  0.   , 137.   ,  40.   , ...,   2.288,  33.   ,   1.   ],
       [  3.   ,  78.   ,  50.   , ...,   0.248,  26.   ,   1.   ],
       ...,
       [  2.   ,  88.   ,  58.   , ...,   0.766,  22.   ,   0.   ],
       [ 10.   , 101.   ,  76.   , ...,   0.171,  63.   ,   0.   ],
       [  5.   , 121.   ,  72.   , ...,   0.245,  30.   ,   0.   ]])

La ejecución del ejemplo muestra que todos los valores de NaN se imputaron correctamente.

